In [2]:
import numpy as np
import pandas as pd

data = pd.read_csv("predictive_maintenance_dataset.csv")
data["date"] = data["date"].apply(lambda col: pd.to_datetime(col, format="%m/%d/%Y"))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124494 entries, 0 to 124493
Data columns (total 12 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   date     124494 non-null  datetime64[ns]
 1   device   124494 non-null  object        
 2   failure  124494 non-null  int64         
 3   metric1  124494 non-null  int64         
 4   metric2  124494 non-null  int64         
 5   metric3  124494 non-null  int64         
 6   metric4  124494 non-null  int64         
 7   metric5  124494 non-null  int64         
 8   metric6  124494 non-null  int64         
 9   metric7  124494 non-null  int64         
 10  metric8  124494 non-null  int64         
 11  metric9  124494 non-null  int64         
dtypes: datetime64[ns](1), int64(10), object(1)
memory usage: 11.4+ MB


In [3]:
data['date_max'] = data.groupby("device")['date'].transform('max')
data['RUL'] = data['date_max'] - data['date']
data["RUL"] = data["RUL"].dt.days
data = data.drop(["date_max", "date"], axis=1)

In [4]:
Failed = data 
Failures = Failed.groupby('device').agg({'failure' : 'max'})
Failures = Failures.rename(columns=({'failure': 'relevant'}))
Failed = Failed.merge(Failures, how='left', on='device')
Failed = Failed[Failed['relevant']==1]
Failed = Failed.reset_index(drop=True)



In [5]:
Failed = Failed.drop(['failure','relevant'], axis=1)
Failed = Failed.drop('device', axis=1)


# Neural Network on RUL value

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [7]:
X_raw = Failed.drop('RUL', axis=1)
y = Failed['RUL']
X = StandardScaler().fit_transform(X_raw)
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [9]:
model_mlp = MLPClassifier(hidden_layer_sizes=(400, 300, 300, 300, 300, 300, 400), max_iter=1000, random_state=42, activation='relu', solver='adam')
model_mlp.fit(X_train, y_train)

# Prédictions sur les données de test
predictions_mlp = model_mlp.predict(X_test)

# Évaluation de l'efficacité du modèle
accuracy_mlp = accuracy_score(y_test, predictions_mlp)
print(f'Test accuracy pour MLPclassifier: {accuracy_mlp}')

ecart = predictions_mlp - y_test
ecart_abs = np.abs(ecart)
print(f'ecart aboslu moyen: {np.mean(ecart_abs)}')
print(f'ecart absolu inférieur à 10: {(ecart_abs < 10).sum() }')

Test accuracy pour MLPclassifier: 0.001866368047779022
ecart aboslu moyen: 30.8006718924972
ecart absolu inférieur à 10: 1062


In [10]:
(ecart_abs < 15).sum() 

1334

# Création de classes

In [25]:
conditions = [(Failed['RUL'] < 7), (Failed['RUL'] < 15), (Failed['RUL'] < 30)]
values = ['Critique', 'Important', 'Modere']
Failed['Criticité'] = np.select(conditions, values, default='Secondaire')
Failed_class = Failed.drop('RUL', axis=1)

# Classification

In [155]:
y = Failed_class['Criticité']
X_train, X_test, y_train, y_test = train_test_split(X,y)

### Random Forest

In [156]:
from sklearn.ensemble import RandomForestClassifier
weights = {'Critique' : 5, 'Important':2, 'Modere':1, 'Secondaire':2}
model_rf = RandomForestClassifier(n_estimators=10, class_weight=weights)
model_rf.fit(X_train, y_train)
from sklearn.metrics import accuracy_score, confusion_matrix
print(f"Le pourcentage de bien classés est de : {accuracy_score(y_test, model_rf.predict(X_test))*100} %")

Le pourcentage de bien classés est de : 88.05524449421426 %


In [157]:
pd.DataFrame(confusion_matrix(y_test, model_rf.predict(X_test)),
             index = ["Critique_données", "Important_données", "Modere_données", "Secondaire_données"],
             columns = ["Critique_predit", "Important_predit", "Modere_predit", "Secondaire_predit"])

,Critique_predit,Important_predit,Modere_predit,Secondaire_predit
Critique_données,120,19,3,19
Important_données,28,94,35,23
Modere_données,2,43,222,74
Secondaire_données,8,13,53,1923


### Neural Network 

In [49]:
MLPClassifier(hidden_layer_sizes=(400, 300, 300, 300, 300, 300, 400), max_iter=1000, random_state=42, activation='relu', solver='adam')
model_mlp.fit(X_train, y_train)

# Prédictions sur les données de test
predictions_mlp = model_mlp.predict(X_test)

# Évaluation de l'efficacité du modèle
accuracy_mlp = accuracy_score(y_test, predictions_mlp)
print(f'Test accuracy pour MLPclassifier: {accuracy_mlp}')

Test accuracy pour MLPclassifier: 0.7353490108249346
